In [1]:
from bs4 import BeautifulSoup 
import re 
import requests
import os
import nltk 
from sklearn import feature_extraction
from gensim import models
import matplotlib
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Embedding, Bidirectional
from keras.callbacks import ModelCheckpoint, LambdaCallback

/Users/daniel/dev/venvs/p36/lib/python3.6/site-packages/sklearn/utils/fixes.py:55: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if 'order' in inspect.getargspec(np.copy)[0]:
/Users/daniel/dev/venvs/p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [ ]:
# Faz uma requisição na página que contém mil links de letras de músicas sertanejas
url_musicas = "https://www.letras.mus.br/mais-acessadas/sertanejo/"
pagina = requests.get(url_musicas)

In [ ]:
# Guarda o conteúdo da página
conteudo = pagina.content

In [ ]:
# Cria o parser HTML
soup = BeautifulSoup(conteudo,"html.parser")

In [ ]:
top_musicas = soup.find("ol",attrs={"class" : "top-list_mus"})

In [ ]:
links_musicas = top_musicas.findAll("a")

In [ ]:
links_musicas[0]

In [ ]:
raiz_site = "https://www.letras.mus.br/"
for link in links_musicas:
    pagina_musica = requests.get(raiz_site + link.get("href"))
    soup_musica = BeautifulSoup(pagina_musica.content,"html.parser")
    artigo_letra = soup_musica.find("article")
    nome_arquivo = str(link.get("href")).replace("/","") + ".html"
    with open("dados/letras-musicas/" + nome_arquivo,"w") as f:
        f.write(str(artigo_letra))

In [2]:
documentos = []
arquivos = os.listdir("dados/letras-musicas")
for arquivo in arquivos:
    caminho = "dados/letras-musicas/" + arquivo
    if os.path.isfile(caminho):
        with open(caminho,"r") as f:
            html = f.read()
        html = feature_extraction.text.strip_accents_ascii(html.lower())
        html = html.replace("<br/>"," NLINHA ")
        html = html.replace("<p>"," NLINHA ")
        soup = BeautifulSoup(html,"html.parser")
        documentos.append(soup.text.strip())

In [ ]:
docs_tokenizados = []
tokenizador = nltk.TreebankWordTokenizer()
for doc in documentos:
    tokens = tokenizador.tokenize(doc)
    docs_tokenizados.append(tokens)

In [ ]:
docs_tokenizados[0]

In [ ]:
# Treinando um modelo Word2vec nos documentos
w2v_model = models.Word2Vec(docs_tokenizados, size=350, window=5, min_count=0, workers=os.cpu_count(),iter=300)

In [ ]:
w2v_model.wv["amor"]

In [ ]:
# Cria um array com o vetor das duas mil primeiras palavras
vocab = list(w2v_model.wv.vocab)
vetores_w2v = w2v_model.wv[vocab]

In [ ]:
import numpy as np
vetores_w2v = np.asfarray(vetores_w2v)

In [ ]:
# Reduzindo a dimensionalidade dos vetores das palavras
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2)
vetores_reduzidos = tsne.fit_transform(vetores_w2v)

In [ ]:
# Cria um dataframe com as colunas x e y. O objetivo é transformar cada representação de uma palavra em uma coordenada.
import pandas as pd
df = pd.DataFrame(vetores_reduzidos, index=vocab, columns=['x', 'y'])
df.head()

In [ ]:
# Utiliza o matplotlib com a chave de interação ligada.
%matplotlib notebook

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)

for word, pos in df.iterrows():
    ax.annotate(word, pos) # Coloca a palavra em cada ponto

ax.scatter(df['x'], df['y']) # Cria um scatterPlot com todos os pontos x e y

In [ ]:
w2v_model.save("modelos/w2v_sertanejo_1000musicas.model")

In [ ]:
w2v_model.wv.vectors.shape

In [ ]:
# Funcoes que retornam o índice da palavra no vocabulário criado pelo word2vec e vice-versa
def word2idx(word):
  return w2v_model.wv.vocab[word].index

def idx2word(idx):
  return w2v_model.wv.index2word[idx]

In [ ]:
maxlen = 5
sentences = []
next_tokens = []

for doc in docs_tokenizados: # Cria as "sentencas" para cada documento do corpus
    for i in range(0,len(doc) - maxlen):
        sentences.append(doc[i:i+maxlen])
        next_tokens.append(doc[i+maxlen])

print('Number of sequences:', len(sentences), "\n")

# sentences = sentences[0:30000] # Limitando em 30 mil sentenças para não demorar...
# next_tokens = next_tokens[0:30000]

In [ ]:
# Criando os inputs(x) e targets(y)
x = np.zeros((len(sentences), maxlen), dtype=np.int32)
y = np.zeros((len(sentences)), dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, token in enumerate(sentence):
        x[i, t] = word2idx(token) 
    y[i] = word2idx(next_tokens[i])

In [ ]:
pretrained_weights = w2v_model.wv.vectors

In [ ]:
tamanho_vocab = pretrained_weights.shape[0]
tamanho_vetor_w2v = pretrained_weights.shape[1] # 350
print("Tamanho vocab e w2v vector: ", (tamanho_vocab, tamanho_vetor_w2v))

In [ ]:
# Definindo o modelo LSTM
model = Sequential()
# A cama de de Embedding é onde podemos passar para a LSTM os vetores já treinados pelo word2vec
model.add(Embedding(input_dim= tamanho_vocab, output_dim=tamanho_vetor_w2v, weights=[pretrained_weights]))
model.add(Bidirectional(LSTM(350, activation="relu", return_sequences=True)))
model.add(Dropout(0.1))
model.add(LSTM(units=280))
# model.add(LSTM(64, input_shape=(pretrained_weights.shape[0], pretrained_weights.shape[1]) ))
# model.add(Dropout(0.1))
model.add(Dense(tamanho_vocab, activation='softmax')) # Quantidade de 'respostas' possíveis. Tokens neste caso. 
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

In [ ]:
# define the checkpoint
caminho_modelo_lstm ="modelos/bilstm-w2v-wordlevel-350-280-sertanejo.model"
checkpoint = ModelCheckpoint(caminho_modelo_lstm, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
def gerar_texto(epoch, logs):
    
    print('----- Generating text after Epoch: %d' % epoch)
    
    # pick a random seed
    start = np.random.randint(0, len(sentences))
    seed_tokens = list(sentences[start])
    print ("Seed:")
    print ("\"", " ".join(seed_tokens), "\"\n")
    
    # generate characters
    for i in range(500):
        
        xt = np.zeros((1, maxlen), dtype=np.int32)
        
        for t, token in enumerate(seed_tokens):
            xt[0,t] = word2idx(token)
        
        prediction = model.predict(xt, verbose=0)
#         ordered = (-prediction[0]).argsort()[:1]
#         print(len(ordered),ordered)
#         for x in ordered:
#             print(prediction[0][x])
#         indice_aleatorio = np.random.choice(ordered)
        index = np.argmax(prediction)
        result = idx2word(index)
        print("\n" if result == "NLINHA" else result,end=" ")
        seed_tokens.append(result)
        seed_tokens = seed_tokens[1:len(seed_tokens)]
        
    print("\n\nFIM\n\n",)

In [ ]:
# Callback para gerar texto de exemplo ao final de cada iteração de treino
print_callback = LambdaCallback(on_epoch_end=gerar_texto)
callbacks_list.append(print_callback)

In [ ]:
model.summary()

In [ ]:
model.fit(x, y, epochs=30, batch_size=64, callbacks=callbacks_list)

In [ ]:
gerar_texto(1,"teste")